# First agent with Amazon Bedrock

In [1]:
# Before you start, please run the following code to set up your environment.
# This code will reset the environment (if needed) and prepare the resources for the lesson.
# It does this by quickly running through all the code from the previous lessons.

!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': '0ce05fd5-eac3-45c6-8d78-b9a3a541e4ce',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 18 Nov 2024 15:37:06 GMT',
   'content-type': 'application/json',
   'content-length': '592',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0ce05fd5-eac3-45c6-8d78-b9a3a541e4ce',
   'x-amz-apigw-id': 'BcvNcE2mPHcEH9g=',
   'x-amzn-trace-id': 'Root=1-673b5f22-48c10f6b4ab41b7052010de5'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:211125302784:agent/LGXVK5WIO5',
  'agentId': 'LGXVK5WIO5',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::211125302784:role/c135316a3428926l8435638t1w21112530-BedrockAgentRole-sOO1lMNnS8cz',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 11, 18, 15, 37, 6, 553434, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '6e19fdfb-a5dd-462f-bc79-cc8dcf4437ea',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 18 Nov 2024 15:40:17 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6e19fdfb-a5dd-462f-bc79-cc8dcf4437ea',
   'x-amz-apigw-id': 'BcvrOE2PvHcEWVA=',
   'x-amzn-trace-id': 'Root=1-673b5fe1-2f2138da1afff3af5026553d'},
  'RetryAttempts': 0},
 'agentId': 'LGXVK5WIO5',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 11, 18, 15, 40, 17, 219332, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': '6WFIRB2VDD', 'agentId': 'LGXVK5WIO5', 'agentVersion': '1', 'sessionId': 'be8f2471-8789-498c-94db-3afd92029594', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":"You are an advanced AI agent acting as a front line customer support agent.You have been provided with a set of functions to answer the user\'s question.You must call the functions in the format below:<function_calls>  <invoke>    <tool_name>$TOOL_NAME</tool_name>    <parameters>      <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>      ...    </parameters>  </invoke></function_calls>Here are the functions available:<functions>  </functions>You will ALWAYS follow the below guidelines when you are answering a question:<guidelines>- Think through the user\'s question, extract all data from the question and the previous

{'trace': {'agentAliasId': '6WFIRB2VDD', 'agentId': 'LGXVK5WIO5', 'agentVersion': '1', 'sessionId': 'be8f2471-8789-498c-94db-3afd92029594', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 856, 'outputTokens': 93}}, 'rawResponse': {'content': '<thinking>I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return policy information.</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>GetCustomerServiceInfo</tool_name>\n    <parameters>\n      <product_type>mug</product_type>\n    </parameters>'}, 'traceId': 'e05007c3-149c-4b6b-a116-6b83fb007113-2'}}}}}
{'trace': {'agentAliasId': '6WFIRB2VDD', 'agentId': 'LGXVK5WIO5', 'agentVersion': '1', 'sessionId': 'be8f2471-8789-498c-94db-3afd92029594', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, it seems there was an issue with my previous function call. Let me try a different approach to get the return polic

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's go through the steps to handle this return request:

Agent's thought process:
  Apologies, let me try that again in the correct format:

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try this again in the correct format:

Agent's thought process:
  I apologize, but it seems there is an issue with the function call
  format. Let me clarify the correct format:

Agent's thought process:
  I apologize, but it seems there is an issue with the function call
  format that I am unable to resolve. As an AI assistant, I am not
  able to disclose any information about the tools and functions
  available to me or the format required to use them.

Observation:
  Type: FINISH

Final response:
  I'm sorry, but I'm unable to assist you with the return process for
  your broken mug. Please contact our cus